### Import Relevant Libraries

In [ ]:
# Libraries
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Preprocessing the Dataset

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Mehta+ML/parsed_ArtOfProblemSolving 3 (3).csv')
data2 = data.drop(columns=['problem_id', 'answer'])

# Final preprocessed data
# Note: some problems are repeated because they contain different solutions

finalset1 = data2.copy()
# Use boolean indexing to filter the DataFrame. Code below removes all problems that are higher than problem 15, removes AJSME problems, and removes AHSME problems
finalset1[finalset1['link'].apply(lambda x: int(x.split('_')[-1]) > 15 or x.split('_')[1] == 'AJHSME' or x.split('_')[1] == 'AHSME')] = np.nan
finalset1 = finalset1.dropna()

'''Problems faced: One challenge was choosing problems that were problem 15 or below. The problem numbers
were a part of the link, and so we had to extract these numbers using the apply method, the lambda function, and the
split method to separate the link components.

Another problem we faced was sorting out only the competitions that we needed, as there were competitions other than
AMC. For this, we had to use an equivalent process as above to extract competition names we didn't want.
'''

from sklearn.model_selection import train_test_split
finalset1_train, finalset1_test = train_test_split(finalset1, test_size=0.2, random_state=42)
finalset1_train.head()


,link,problem,solution,letter
71,https://artofproblemsolving.com/wiki/index.php...,"Lola, Lolo, Tiya, and Tiyo participated in a p...","In total, there will be $\binom{4}{2} \cdot 2 ...",A
1968,https://artofproblemsolving.com/wiki/index.php...,You are playing a game. A $2 \times 1$ rectang...,We realize that every $2 \times 1$ rectangle m...,C
5484,https://artofproblemsolving.com/wiki/index.php...,A ship sails $10$ miles in a straight line fro...,Let $C_1$ be the point the ship would reach if...,D
70,https://artofproblemsolving.com/wiki/index.php...,"Lola, Lolo, Tiya, and Tiyo participated in a p...",We can calculate the total number of wins ( $1...,A
4087,https://artofproblemsolving.com/wiki/index.php...,Positive real numbers $x$ and $y$ satisfy $y^3...,We will use the difference of squares in the s...,D


In [ ]:
finalset1_train_half = finalset1_train.copy()

for i in finalset1_train_half.index:
  finalset1_train_half.loc[i, "instruction"] = "Solve the math problem below."
  finalset1_train_half.loc[i, "input"] = f"{finalset1_train_half.loc[i, 'problem']}"
  finalset1_train_half.loc[i, "output"] = f"{finalset1_train_half.loc[i, 'solution']}. So the correct answer choice is {finalset1_train_half.loc[i, 'letter']}."

finalset1_train_half = finalset1_train_half.drop(columns=['link', 'problem', 'solution', 'letter'])
finalset1_train_half

,instruction,input,output
71,Solve the math problem below.,"Lola, Lolo, Tiya, and Tiyo participated in a p...","In total, there will be $\binom{4}{2} \cdot 2 ..."
1968,Solve the math problem below.,You are playing a game. A $2 \times 1$ rectang...,We realize that every $2 \times 1$ rectangle m...
5484,Solve the math problem below.,A ship sails $10$ miles in a straight line fro...,Let $C_1$ be the point the ship would reach if...
70,Solve the math problem below.,"Lola, Lolo, Tiya, and Tiyo participated in a p...",We can calculate the total number of wins ( $1...
4087,Solve the math problem below.,Positive real numbers $x$ and $y$ satisfy $y^3...,We will use the difference of squares in the s...
...,...,...,...
4039,Solve the math problem below.,"In the year $2001$ , the United States will ho...","First, we need to recognize that a number is g..."
2956,Solve the math problem below.,A rectangular box has integer side lengths in ...,Let the smallest side length be $x$ . Then the...
3033,Solve the math problem below.,How many squares whose sides are parallel to t...,The region is a right triangle which contains ...
3369,Solve the math problem below.,Last summer 30% of the birds living on Town La...,"WLOG, suppose there were 100 birds in total li..."


In [ ]:
finalset2_test_features = finalset1_test.copy()
finalset2_test_features.drop(columns=['link'])

for i in finalset2_test_features.index:
  finalset2_test_features.loc[i, "instruction"] = f"Solve the math problem below."
  finalset2_test_features.loc[i, "input"] = f"{finalset2_test_features.loc[i, 'problem']}"
  finalset2_test_features.loc[i, "output"] = ""
finalset2_test_features = finalset2_test_features.drop(columns=['link', 'problem', 'solution', 'letter'])
finalset2_test_features = finalset2_test_features.drop_duplicates()
finalset2_test_features = finalset2_test_features.iloc[:250]

finalset2_test_labels = finalset1_test.copy()
for i in finalset2_test_labels.index:
  finalset2_test_labels.loc[i, "instruction"] = f"Solve the math problem below."
  finalset2_test_labels.loc[i, "input"] = f"{finalset2_test_labels.loc[i, 'problem']}"
  finalset2_test_labels.loc[i, "output"] = f"{finalset1_test.loc[i, 'letter']}"
finalset2_test_labels = finalset2_test_labels.drop(columns=['link', 'problem', 'solution', 'letter'])
finalset2_test_labels = finalset2_test_labels.drop_duplicates()
finalset2_test_labels = finalset2_test_labels.drop(columns=['instruction', 'input'])

finalset2_test_labels = finalset2_test_labels.iloc[:250]
finalset2_test_features


,instruction,input,output
3747,Solve the math problem below.,A scout troop buys $1000$ candy bars at a pric...,
4413,Solve the math problem below.,What is the least possible value of $(xy-1)^2+...,
390,Solve the math problem below.,The number $N$ is a two-digit number.\n• When ...,
5634,Solve the math problem below.,How many even three-digit integers have the pr...,
5452,Solve the math problem below.,"Let $a$ $b$ $c$ $d$ , and $e$ be positive inte...",
...,...,...,...
476,Solve the math problem below.,Isabella had a week to read a book for a schoo...,
5626,Solve the math problem below.,"A football game was played between two teams, ...",
3728,Solve the math problem below.,A wooden cube $n$ units on a side is painted r...,
711,Solve the math problem below.,"Problems 14, 15 and 16 involve Mrs. Reed's Eng...",


### Preparing Fine-tuning

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",          # Phi-3 2x faster!d
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "", # Enter any model name
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

Unsloth unsuccessfully patched LoraLayer.update_layer. Please file a bug report.
Luckily, your training run will still work in the meantime!


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch Attention layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch O projection layer with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2024.7 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Already have LoRA adapters! We shall skip this step.


### Data Prep


In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
#dataset = load_dataset("parquet", data_files={'train': '/content/drive/MyDrive/Mehta+ML/finalset1_train_half.parquet'})
#dataset = dataset.map(formatting_prompts_func, batched = True,)

<a name="Train"></a>
### Train the model


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    #train_dataset = dataset['train'],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        #max_steps = 400,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
5.984 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,088 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 261
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.318200
2,1.351400
3,1.418700
4,1.308300
5,1.293500
6,1.297200
7,1.010600
8,0.852900
9,0.857600
10,0.772800


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

2779.7886 seconds used for training.
46.33 minutes used for training.
Peak reserved memory = 10.023 GB.
Peak reserved memory for training = 4.039 GB.
Peak reserved memory % of max memory = 67.962 %.
Peak reserved memory for training % of max memory = 27.387 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [ ]:
df = pd.DataFrame(columns=['response', 'letter', 'correct_letter'])

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
count = 0
checkpoint = 0
for i in finalset2_test_features.index:
  inputs = tokenizer(
  [
      alpaca_prompt.format(
          "Give the correct letter-answer to the math problem below. No yapping; just the answer.", # instruction
          f"{finalset2_test_features.loc[i, 'input']}", # input
          "", # output - leave this blank for generation!
      )
  ], return_tensors = "pt").to("cuda")

  outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
  answer = tokenizer.batch_decode(outputs)
  answer_check = answer[0].split()[-1][0]
  df.loc[-1] = [answer, answer_check, finalset2_test_labels.loc[i, 'output']]  # adding a row
  df.index = df.index + 1
  csv_data = df.to_csv('File name to save responses', index = True)
  if answer_check == finalset2_test_labels.loc[i, 'output']:
    count += 1

  checkpoint +=1
  print(checkpoint)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250


In [ ]:
d = pd.read_csv('File name of responses file')
d

,Unnamed: 0,response,letter,correct_letter
0,249,"[""Below is an instruction that describes a tas...",C,A
1,248,['Below is an instruction that describes a tas...,$,D
2,247,['Below is an instruction that describes a tas...,$,E
3,246,['Below is an instruction that describes a tas...,E,B
4,245,['Below is an instruction that describes a tas...,$,B
...,...,...,...,...
245,4,['Below is an instruction that describes a tas...,D,B
246,3,['Below is an instruction that describes a tas...,A,A
247,2,['Below is an instruction that describes a tas...,$,B
248,1,['Below is an instruction that describes a tas...,E,C


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Give the correct letter-answer to the math problem below. No yapping; just the answer.", # instruction
        "What is the least possible value of $(xy-1)^2+(x+y)^2$ for real numbers $x$ and $y$ $\textbf{(A)} ~0\qquad\textbf{(B)} ~\frac{1}{4}\qquad\textbf{(C)} ~\frac{1}{2} \qquad\textbf{(D)} ~1 \qquad\textbf{(E)} ~2$", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
answer = tokenizer.batch_decode(outputs)
answer

['Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nGive the correct letter-answer to the math problem below. No yapping; just the answer.\n\n### Input:\nWhat is the least possible value of $(xy-1)^2+(x+y)^2$ for real numbers $x$ and $y$ $\textbf{(A)} ~0\\qquad\textbf{(B)} ~\x0crac{1}{4}\\qquad\textbf{(C)} ~\x0crac{1}{2} \\qquad\textbf{(D)} ~1 \\qquad\textbf{(E)} ~2$\n\n### Response:\nThe least value occurs when $x=y=0$, so the answer is $\\boxed{0}$. So the correct answer choice is A.<|end_of_text|>']

### Extracting Answers From Mistral 7B's and Llama 3.1 8B's responses

In [ ]:
import pandas as pd
from ast import literal_eval
import numpy as np

input_str = "Input:"
response_str = "Response:"
def get_question(string_list):
    global input_str
    global response_str
    if isinstance(string_list, list):
        string = string_list[0]
    else:
        string = string_list
    try:
        idx1 = string.index(input_str)
        idx2 = string.index(response_str)
        string = string[idx1 + len(input_str) : idx2].replace("\n", "").replace("\\n", "").strip()
        # print(string)
        return string
    except:
        return string

def get_correct(filepath):
    df = pd.read_csv(filepath)
    df = df.dropna().reset_index()
    df.loc[:, "Question"] = df.response.apply(get_question)
    # print(df.shape)
    df = df.drop_duplicates(subset=["Question"])
    # print(df.Question)
    # print(df.shape)
    df.response = df.response.apply(literal_eval)
    responses = list(df["response"].values)
    for response in responses:
        if len(response) != 1:
            print(response)

    response_start = "Response:\n"
    trunc_answers = []
    trunc_responses = []
    eliminate_idx  = []
    index_answer_dict = {}
    for idx, response in enumerate(responses):
        response = response[0]
        val2 = response.index(response_start)
        trunc_responses.append(response[val2 + len(response_start):])
        try:
            x = re.findall(r'\$\\.+A.+B.+C.+D.+E.+\$', response)[-1]
        except:
            eliminate_idx.append(idx)
            continue
        if x.endswith("\\qquad$"):
            x = x[:-len("\\qquad")] + "$"
        x = x.replace("\\quad", "\\qquad")[1:-1].split("\\qquad")
        if len(x) == 1:
            index_answer_dict[idx] = {"A": "42", "B": "63", "C": "84", "D": "126", "E": "252"} # Hardcoded one exception
            continue
        answer_dict = {}
        for idx2, letter in enumerate(["A", "B", "C", "D", "E"]):
            gp = re.findall(r"\\.*" + letter + r".*?}", x[idx2].strip())[0]
            val = x[idx2].replace(gp, "") \
                        .replace("\\ ", "") \
                        .replace("\\mbox{ }", "") \
                        .replace("\\; ", "") \
                        .replace("\\: ", "") \
                        .replace("\\, ", "") \
                        .replace("\\hspace{.05in}", "") \
                        .replace("\\textdollar ", "") \
                        .replace("~", "") \
                        .replace("{{{", "") \
                        .replace("}}}", "") \
                        .replace("\\text{ dollars}", "") \
                        .replace("$ ", "") \
                        .strip()
            if val.endswith("\\"):
                val = val[:-1]
            answer_dict[letter] = val
        index_answer_dict[idx] = answer_dict
    df2 = df.copy()
    for i in df2.index:
      if df2.loc[i, 'letter'] not in ['A', 'B', 'C', 'D', 'E'] or i in eliminate_idx:
        df2 = df2.drop(index=i)

    straightforward_num = (df2["letter"] == df2["correct_letter"]).sum()

    not_found = 0
    answers = []
    not_found_responses = []
    for idx, response in enumerate(trunc_responses):
        if idx in eliminate_idx:
            answers.append("X")
            continue
        groups = re.findall("correct answer choice is (.)", response)
        if groups:
            groups = list(filter(lambda x: x in ["A", "B", "C", "D", "E"], groups))
            answers.append(groups[0]) # all groups ONLY have one option at this point
            continue
        found = False
        for key, val in index_answer_dict[idx].items():
            if val in response:
                answers.append(key)
                found = True
                break
        if found:
            continue
        # not_found_responses.append((response, index_answer_dict[idx].values(), df["correct_letter"][idx]))
        not_found += 1
        answers.append("X")

    # print(not_found, not_found_responses)
    fancy_num = np.sum(list(map(lambda a: a[0] == a[1], zip(list(answers), df["correct_letter"]))))

    valid_answers = len(answers) - len(eliminate_idx)
    print("Valid Answers:", valid_answers)
    print("Explicit:", straightforward_num, "Explicit Ratio:", straightforward_num/valid_answers)
    print("Explicit + Implicit:", fancy_num, "Explicit + Implicit Ratio:", fancy_num/valid_answers)

<a name="Save"></a>
### Saving, loading finetuned models


In [ ]:
model.save_pretrained("Model name") # Local saving
tokenizer.save_pretrained("Model name")

('/content/drive/MyDrive/Mehta+ML/lora_model/tokenizer_config.json',
 '/content/drive/MyDrive/Mehta+ML/lora_model/special_tokens_map.json',
 '/content/drive/MyDrive/Mehta+ML/lora_model/tokenizer.json')

In [ ]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "Model name", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


/usr/local/lib/python3.10/dist-packages/accelerate/utils/modeling.py:1381: UserWarning: Current model requires 104858368.0 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [ ]:
if True:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "Model name", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("Model name")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
